In [5]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from matplotlib import rcParams
import sympy as sy
import scipy.constants as pc
from sympy import *
from math import log10, floor
from numpy import sqrt

In [14]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)], zMaxArray[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (unsicherheiten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def einfachGaussMultMax(unsicherheiten, werte, exponenten):
    max_a = 0
    num = -1
    for i in range (unsicherheiten.size):
        maxi = (exponenten[i]*unsicherheiten[i]/werte[i])**2
        if max_a <= maxi:
            max_a = maxi
            num = i
    return (max_a, num)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]-(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

def round_to_j(x,j): #signifikante stellem
    return round(x, (j-1)-int(floor(log10(abs(x)))))

def rounda_to_j(x,j): #signifikante stellen
    for i in range(len(x)):
        x[i] = round_to_j(x[i],j)
    return x

def rounda_to_m(x,m):
    for i in range(len(x)):
        x[i] = round(x[i],m)
    return x

In [15]:
beta_rech = np.linspace(0.2, 5, 100)
gf_rech = 1+beta_rech**(-1)
bf_rech = beta_rech +1
ef_rech = 2+ beta_rech+ beta_rech**(-1)
#fig, ax = plt.subplots(figsize=(10,6))
#ax.tick_params(axis='both', which='major', labelsize=18)
#ax.tick_params(axis='both', which='minor', labelsize=16)
#ax.set_xscale('log')

#ax.plot(beta_rech, bf_rech, color='r', label=r'$\frac{b}{f}$')
#ax.plot(beta_rech, gf_rech, color='g', label=r'$\frac{g}{f}$ ')
#ax.plot(beta_rech, ef_rech, color='b', label=r'$\frac{e}{f}$ ')

#ax.set(xlabel=r'$\beta$', ylabel=r'Jeweilige Abhängigkeit', title=r'$\frac{b}{f}(\beta), \frac{g}{f}(\beta), \frac{e}{f}(\beta)$')
#ax.legend()
#ax.grid()

#fig.savefig("AbhBeta.svg")

In [16]:
# Gemessene Werte Versuchteil 1
pos_dia = 200 #+- 1
e_gem_1 = np.array([647, 585, 552, 535, 522, 520, 521, 521, 525, 531, 536, 543, 550]) - pos_dia #sqrt(1^2+0.5^2)=sqrt(1.25)
g_gem_1 = np.array([300,310,320,330,340,350,360,370,380,390,400,410,420]) - pos_dia#<-- nice +-1
b_gem_1 = e_gem_1-g_gem_1

e_gem_1_f, g_gem_1_f = 2, 1
b_gem_1_f = np.sqrt(e_gem_1_f**2+g_gem_1_f**2) #Ich hab das mit der Formel gemacht, das geht so!

f_gem_1 = g_gem_1*b_gem_1/(g_gem_1_f+b_gem_1) #Brennweite

beta_gem_1 = b_gem_1/g_gem_1
beta_gem_1_f = (1/g_gem_1)*(np.sqrt((e_gem_1_f**2)+((e_gem_1/g_gem_1)**2)*(g_gem_1_f**2)))

#Nominelle Werte
f_nom_1 = 80 
f_nom_1_f = 0.1 
bf_nom_1 = (1-(f_nom_1/g_gem_1))**(-1)
bf_nom_1_f = einfachGaussMult(np.array([b_gem_1_f, g_gem_1_f]), np.array([b_gem_1, g_gem_1]), np.array([1, 1]))
gf_nom_1 = (1-(f_nom_1/b_gem_1))**(-1)
gf_nom_1_f = einfachGaussMult(np.array([b_gem_1_f,g_gem_1_f]), np.array([b_gem_1,g_gem_1]), np.array([1,1]))
ef_nom_1 = (((1/f_nom_1)-(1/g_gem_1))**(-1)+((1/f_nom_1)-(1/b_gem_1))**(-1))/(f_nom_1)
ef_nom_1_f = sqrt(bf_nom_1_f**2+gf_nom_1_f**2)


print(rounda_to_m(bf_nom_1,3))
print(rounda_to_m(gf_nom_1,3))
print(rounda_to_m(ef_nom_1,3))
print()








beta_rech = np.linspace(0.6, 3.5, 100)
gf_rech = 1+beta_rech**(-1)
bf_rech = beta_rech +1
ef_rech = 2+ beta_rech+ beta_rech**(-1)


f_gem_1 = am((g_gem_1/e_gem_1)*(e_gem_1-g_gem_1))
f_gem_1_f = am(np.sqrt((((g_gem_1-b_gem_1)**2)/(e_gem_1**2))*(e_gem_1_f**2)+(((g_gem_1/e_gem_1)**4)*(b_gem_1_f**2))+((b_gem_1/e_gem_1)**4)*(g_gem_1_f**2)))
phi_gem_1 = 1/f_gem_1
phi_gem_1_f = phi_gem_1*(f_gem_1_f/f_gem_1)
#print('f gemessen, fehler',f_gem_1, f_gem_1_f)
#print('phi gemessen, fehler', phi_gem_1, phi_gem_1_f)

[5.    3.667 3.    2.6   2.333 2.143 2.    1.889 1.8   1.727 1.667 1.615
 1.571]
[1.3   1.41  1.526 1.64  1.784 1.889 1.988 2.127 2.231 2.311 2.429 2.509
 2.6  ]
[6.3   5.077 4.526 4.24  4.118 4.032 3.988 4.016 4.031 4.039 4.095 4.125
 4.171]



In [ ]:
# Erinnerungen
# x = np.array([,])
# x.size
# x, y = symbols('x y')
# expr = x*y
# expr.subs([(x, wert_x), (y, wert_y)])

In [31]:
# Gemessene Werte Versuchteil 2
g_gem_2 = np.array([300,310,320,330,340,350,360,370]) - pos_dia  
e_gem_2_unv = np.array([432,426,427,430,435,440,446,452]) - pos_dia #Unvertauschte Linsen
e_gem_2_ver = np.array([450,431,425,424,430,435,440,445]) - pos_dia #Vertauschte Linsen

d_2, d_2_f = 20, 1

g_gem_2_f, e_gem_2_f = 1, 2
b_gem_2_f = np.sqrt(e_gem_2_f**2+g_gem_2_f**2)

b_gem_2_unv = e_gem_2_unv-g_gem_2
b_gem_2_ver = e_gem_2_ver-g_gem_2

f_gem_2_unv = g_gem_2*b_gem_2_unv/(g_gem_2+b_gem_2_unv)
f_fem_2_ver = g_gem_2*b_gem_2_ver/(g_gem_2+b_gem_2_ver)

#BETAS

beta_gem_2_unv = b_gem_2_unv/g_gem_2
beta_gem_2_unv_f = (1/g_gem_2)*(np.sqrt((e_gem_2_f**2)+((e_gem_2_unv/g_gem_2)**2)*(g_gem_2_f**2)))

beta_gem_2_ver = b_gem_2_ver/g_gem_2
beta_gem_2_ver_f = (1/g_gem_2)*(np.sqrt((e_gem_2_f**2)+((e_gem_2_ver/g_gem_2)**2)*(g_gem_2_f**2)))


# F LINSENSYSTEM

f_1, f_2 = 80, 150
f_nom_2 = ((1/(f_1))+(1/(f_2)))**(-1)#-d_2/(f_1+f_2)**(-1)

# NOMINELLE DATEN

bf_nom_2_unv = (1-(f_nom_2/g_gem_2))**(-1)
gf_nom_2_unv = (1-(f_nom_2/b_gem_2_unv))**(-1) 
ef_nom_2_unv = (((1/f_nom_2)-(1/g_gem_2))**(-1)+((1/f_nom_2)-(1/b_gem_2_unv))**(-1))/(f_nom_2)

bf_nom_2_unv_f = einfachGaussMult(np.array([b_gem_2_f,g_gem_2_f]), np.array([b_gem_2_unv,g_gem_2]), np.array([1,1])) #bf_nom_2_unv*(g_gem_2_f/g_gem_2)
gf_nom_2_unv_f = einfachGaussMult(np.array([b_gem_2_f,g_gem_2_f]), np.array([b_gem_2_unv,g_gem_2]), np.array([1,1])) #gf_nom_2_unv*(b_gem_2_f/b_gem_2_unv)
ef_nom_2_unv_f = sqrt(bf_nom_2_unv_f**2+gf_nom_2_unv_f**2)# ef_nom_2_unv*(am([(b_gem_2_f/b_gem_2_unv), (g_gem_2_f/g_gem_2)]))

print(rounda_to_m(bf_nom_2_unv,2))
print(rounda_to_m(gf_nom_2_unv,2))
print(rounda_to_m(ef_nom_2_unv,2))
#print()
print(rounda_to_j(bf_nom_2_unv_f,1))
print(rounda_to_j(gf_nom_2_unv_f,1))
print(rounda_to_j(ef_nom_2_unv_f,1))
print()
print()
print()
print()





bf_nom_2_ver = (1-(f_nom_2/g_gem_2))**(-1)
gf_nom_2_ver = (1-(f_nom_2/b_gem_2_ver))**(-1) 
ef_nom_2_ver = (((1/f_nom_2)-(1/g_gem_2))**(-1)+((1/f_nom_2)-(1/b_gem_2_ver))**(-1))/(f_nom_2)

bf_nom_2_ver_f = einfachGaussMult(np.array([b_gem_2_f,g_gem_2_f]), np.array([b_gem_2_ver,g_gem_2]), np.array([1,1])) #bf_nom_2_ver*(g_gem_2_f/g_gem_2)
gf_nom_2_ver_f = einfachGaussMult(np.array([b_gem_2_f,g_gem_2_f]), np.array([b_gem_2_ver,g_gem_2]), np.array([1,1])) #gf_nom_2_ver*(b_gem_2_f/b_gem_2_ver)
ef_nom_2_ver_f = sqrt(bf_nom_2_ver_f**2+gf_nom_2_ver_f**2) #ef_nom_2_ver*(am([(b_gem_2_f/b_gem_2_ver), (g_gem_2_f/g_gem_2)]))

print(rounda_to_m(bf_nom_2_ver,3))
print(rounda_to_m(gf_nom_2_ver,3))
print(rounda_to_m(ef_nom_2_ver,3))
#print()
#print(rounda_to_j(bf_nom_2_ver_f,1))
#print(rounda_to_j(gf_nom_2_ver_f,1))
#print(rounda_to_j(ef_nom_2_ver_f,1))
#print()
#print(bf_nom_2_ver_f)
#print(bf_nom_2_ver_f)
#print(bf_nom_2_ver_f)


#
# PLOTS
# GEMESSENEN WERTE







b_gem_2 = (b_gem_2_unv+b_gem_2_ver)/2
e_gem_2 = (e_gem_2_unv+e_gem_2_ver)/2

f_gem_2 = am((g_gem_2/((e_gem_2_unv+e_gem_2_ver)/2))*(((e_gem_2_unv+e_gem_2_ver)/2)-g_gem_2))
f_gem_2_f = am(np.sqrt((((g_gem_2-b_gem_2)**2)/(e_gem_2**2))*(e_gem_2_f**2)+(((g_gem_2/e_gem_2)**4)*(b_gem_2_f**2))+((b_gem_2/e_gem_2)**4)*(g_gem_2_f**2)))
phi_gem_2 = 1/f_gem_2
phi_gem_2_f = phi_gem_2*(f_gem_2_f/f_gem_2)
#print('f gemessen, fehler',f_gem_2, f_gem_2_f)
#print('phi gemessen, fehler', phi_gem_2, phi_gem_2_f)

[2.09 1.9  1.77 1.67 1.59 1.53 1.48 1.44]
[1.65 1.82 1.95 2.09 2.22 2.38 2.54 2.75]
[3.74 3.72 3.72 3.76 3.81 3.91 4.03 4.19]
[0.02 0.02 0.02 0.02 0.02 0.03 0.03 0.03]
[0.02 0.02 0.02 0.02 0.02 0.03 0.03 0.03]
[0.03 0.03 0.03 0.03 0.03 0.04 0.04 0.04]




[2.091 1.902 1.769 1.67  1.594 1.533 1.484 1.443]
[1.533 1.758 1.988 2.247 2.379 2.589 2.875 3.286]
[3.624 3.66  3.757 3.918 3.973 4.123 4.359 4.729]


In [ ]:
#Gemessene Werte


g_gem_3 = np.array([400,410,420,430,440,450,460,470])  
e_gem_3_unv = np.array([732,704,685,674,667,665,664,653]) #Unvertauschte Linsen
e_gem_3_ver = np.array([662,661,661,663,669,673,679,681]) #Vertauschte Linsen
Bg_gem_3_unv = np.array([15,12,11,10,8,8,7,6])
Bg_gem_3_ver = np.array([8,7,6,6,6,5,5,5])
print(g_gem_3)
print(e_gem_3_unv)
print(e_gem_3_ver)
print(Bg_gem_3_unv)
print(Bg_gem_3_ver)


In [ ]:
#Gemessene Werte 

f_4 = 250
f_gem_4_blau = 438
f_gem_4_rot = 446

